<a href="https://colab.research.google.com/github/marcleipold/Project-2---Algo-Trading-ML-Bot/blob/marc/bitcoin_tweets_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing all of the libraries

import numpy as np
import pandas as pd
import os
from tqdm import tqdm

#For Preprocessing & Vader Sentiment Analysis
import re    # RegEx for removing non-letter characters
import nltk  # natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [1]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_tweets.csv to cleaned_tweets (1).csv


In [3]:
# Load Tweet dataset
df0 = pd.read_csv('cleaned_tweets.csv', index_col='date', parse_dates=True, infer_datetime_format=True)
df0

,text
date,
2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...
2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""..."
2021-02-10 23:54:48,"Guys evening, I have read this article about B..."
2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...
2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...
...,...
2021-09-10 21:15:34,@SpeedUpBSC @debi_pada @AdaTiers @golf_grinder...
2021-09-10 21:15:33,"This time, what will happen between the US SEC..."
2021-09-10 21:15:04,One #btc is only 205 #EGLD.


In [17]:
# selecting the rows for the analysis

df=df0[0:1000000]

df

,text
date,
2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...
2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""..."
2021-02-10 23:54:48,"Guys evening, I have read this article about B..."
2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...
2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...
...,...
2021-08-15 12:53:23,#wtfhappenedin1971? \n#Bitcoin will fix this!
2021-08-15 12:53:21,@CryptoNadine @BabyArabiaBSC #babyarabia massi...
2021-08-15 12:53:12,@mcuban Are you serious or part of the scam??\...


In [18]:
# Tokenizing the tweets by breaking them down into separate words

def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
     
    # Check if the input is a string
    if not isinstance(tweet, str):
        raise TypeError("Input is not a string")
    
    # convert to lowercase
    text = tweet.lower() 
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words


In [19]:
# cleaning the text by using the function from above

cleantext=[]
for item in tqdm(df['text']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleantext']=cleantext
df

100%|██████████| 1000000/1000000 [1:08:39<00:00, 242.75it/s]
<ipython-input-19-d254c76d8549>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleantext']=cleantext


,text,cleantext
date,,
2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,"[blue, ridg, bank, share, halt, nyse, bitcoin,..."
2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","[today, thursday, take, 2, friend, leowandersl..."
2021-02-10 23:54:48,"Guys evening, I have read this article about B...","[guy, even, read, articl, btc, would, like, sh..."
2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"[btc, big, chanc, billion, price, 4872644, 0, ..."
2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,"[network, secur, 9, 508, node, today, soon, bi..."
...,...,...
2021-08-15 12:53:23,#wtfhappenedin1971? \n#Bitcoin will fix this!,"[wtfhappenedin1971, bitcoin, fix]"
2021-08-15 12:53:21,@CryptoNadine @BabyArabiaBSC #babyarabia massi...,"[cryptonadin, babyarabiabsc, babyarabia, massi..."
2021-08-15 12:53:12,@mcuban Are you serious or part of the scam??\...,"[mcuban, seriou, part, scam, scam, bsv, bitcoi..."


In [24]:
# viewing the new dataframe (df)
df

,text,cleantext
date,,
2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,"[blue, ridg, bank, share, halt, nyse, bitcoin,..."
2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","[today, thursday, take, 2, friend, leowandersl..."
2021-02-10 23:54:48,"Guys evening, I have read this article about B...","[guy, even, read, articl, btc, would, like, sh..."
2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"[btc, big, chanc, billion, price, 4872644, 0, ..."
2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,"[network, secur, 9, 508, node, today, soon, bi..."
...,...,...
2021-08-15 12:53:23,#wtfhappenedin1971? \n#Bitcoin will fix this!,"[wtfhappenedin1971, bitcoin, fix]"
2021-08-15 12:53:21,@CryptoNadine @BabyArabiaBSC #babyarabia massi...,"[cryptonadin, babyarabiabsc, babyarabia, massi..."
2021-08-15 12:53:12,@mcuban Are you serious or part of the scam??\...,"[mcuban, seriou, part, scam, scam, bsv, bitcoi..."


In [25]:
# concatenating items in a list to create 

def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [26]:
# compute Vader Sentiment analysis to compute the polarity of the tweets

def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["vader_neg"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["vader_neu"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["vader_pos"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["vader_comp"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    df['cleantext2'] = df[label].apply(lambda x: unlist(x))
    return df

In [27]:
# add Vader score to a new dataframe and reset index

df2 = compute_vader_scores(df, 'cleantext')
df2 = df2.reset_index()




<ipython-input-26-16c4c2aed94e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["vader_neg"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
<ipython-input-26-16c4c2aed94e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["vader_neu"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
<ipython-input-26-16c4c2aed94e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [28]:
df2

,date,text,cleantext,vader_neg,vader_neu,vader_pos,vader_comp,cleantext2
0,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,"[blue, ridg, bank, share, halt, nyse, bitcoin,...",0.000,0.872,0.128,0.2960,blue ridg bank share halt nyse bitcoin atm ann...
1,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","[today, thursday, take, 2, friend, leowandersl...",0.000,0.775,0.225,0.4939,today thursday take 2 friend leowandersleb btc...
2,2021-02-10 23:54:48,"Guys evening, I have read this article about B...","[guy, even, read, articl, btc, would, like, sh...",0.000,0.719,0.281,0.5719,guy even read articl btc would like share http...
3,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"[btc, big, chanc, billion, price, 4872644, 0, ...",0.000,1.000,0.000,0.0000,btc big chanc billion price 4872644 0 2021 02 ...
4,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,"[network, secur, 9, 508, node, today, soon, bi...",0.200,0.800,0.000,-0.5423,network secur 9 508 node today soon biggest be...
...,...,...,...,...,...,...,...,...
999995,2021-08-15 12:53:23,#wtfhappenedin1971? \n#Bitcoin will fix this!,"[wtfhappenedin1971, bitcoin, fix]",0.000,1.000,0.000,0.0000,wtfhappenedin1971 bitcoin fix
999996,2021-08-15 12:53:21,@CryptoNadine @BabyArabiaBSC #babyarabia massi...,"[cryptonadin, babyarabiabsc, babyarabia, massi...",0.000,0.738,0.262,0.8316,cryptonadin babyarabiabsc babyarabia massiv bi...
999997,2021-08-15 12:53:12,@mcuban Are you serious or part of the scam??\...,"[mcuban, seriou, part, scam, scam, bsv, bitcoi...",0.346,0.654,0.000,-0.8126,mcuban seriou part scam scam bsv bitcoin need ...
999998,2021-08-15 12:52:53,Why is #Bitcoin going to ascend to a global re...,"[bitcoin, go, ascend, global, reserv, currenc,...",0.182,0.818,0.000,-0.5994,bitcoin go ascend global reserv currenc bring ...


In [29]:
# convert the 'date_str' column to datetime, replacing invalid strings with NaT
df2['date2'] = pd.to_datetime(df2['date'], errors='coerce')

# drop the rows with NaT in the 'date' column
df2.dropna(subset=['date2'], inplace=True)

# reset the index
df2.reset_index(drop=True, inplace=True)

In [30]:


# group by date and aggregate the values for the Vader comp score
agg_df = df2.groupby(by=df2['date2'].dt.date).agg({'vader_comp': 'mean'})

agg_df



,vader_comp
date2,
2021-02-05,0.085806
2021-02-06,0.098810
2021-02-07,0.096075
2021-02-08,0.124207
2021-02-09,0.113830
...,...
2021-08-17,0.231158
2021-08-18,0.263905
2021-08-19,0.228848


In [31]:
agg_df.to_csv("bitcoin-tweets-vader-sentiment-final.csv", index=True)